In [0]:
import networkx as nx
from enum import Enum
import numpy as np
import copy
import random
import matplotlib.pyplot as plt
import pandas as pd
import pickle


In [0]:
############## params of graph ############################
# N = 1000
# M = 20
# PROB_TRIANGLE = 0.8
###########################################################

# g = nx.newman_watts_strogatz_graph(N, M, PROB_TRIANGLE, seed=42) ########### toy data #######

# len(nx.triangles(g))
# print(g.number_of_edges())

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
os.chdir('/content/gdrive/My Drive/antivax_data/')

In [0]:
# g = nx.read_edgelist("twitter_combined.txt", create_using = nx.Graph(), nodetype=int) ########### snap twitter data #############
 
g = nx.read_edgelist("facebook_combined.txt", create_using = nx.Graph(), nodetype=int) ########### snap facebook data #############

# d = 5
# g = nx.random_regular_graph(d=d, n = N, seed = 42)

In [0]:
# nx.draw(g)

In [0]:
def plot(x, y, label, title, xlabel, ylabel, showplot = True):
    # plot
    plt.plot(x, y, label=label)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    if(showplot):
      plt.show()

In [0]:
def save_obj(obj, name ):
    with open('obj/'+ name + '.pkl', 'wb+') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)




Start simulation




In [0]:
##############  first define the utility function #################

def get_util(graph, i, x, state, alpha, beta, C, gamma, delta):
  util = 0

  for v in graph.neighbors(i):
    if x[v] == state:
      # print(v, " same as ", i)
      util += alpha
    else:
      # print(v, " different from ", i)
      util -= beta

  if(sum(x.values()) < (1-gamma)*graph.number_of_nodes()):
    util += delta

  elif state == 0:
    util += C
  
  return util


In [0]:
def simulate_one_parallel_epoch(graph, x, alpha, beta, gamma, C, delta, ):
  new_x = copy.deepcopy(x)
  for i in graph.nodes:
    if(get_util(graph=graph, i=i, x=x, state = x[i], alpha = alpha, beta = beta, C = C, gamma = gamma, delta = delta) 
    < get_util(graph = graph, i = i, x = x, state= 1-x[i], alpha = alpha, beta = beta, gamma = gamma, C = C, delta = delta)): 
      # new_x[i] = (x[i] == SENTIMENT.PRO.value) and SENTIMENT.ANTI.value or SENTIMENT.PRO.value
      new_x[i] = 1-x[i]

  return new_x

In [0]:
def simulate_parallel(graph, strategy, alpha, beta, gamma, C, delta, verbose, TMAX =50):
  t = 0
  print("antivax at time ", t, ": ", sum(strategy.values())," Probability: ", sum(strategy.values()) / len(g.nodes) )
  
  new_strategy = simulate_one_parallel_epoch(graph = graph, x = strategy, alpha = alpha, beta = beta, gamma=gamma, C = C, delta = delta)

  antivax_history = [sum(strategy.values())]

  while(new_strategy != strategy and t < TMAX):
    t += 1
    strategy = copy.deepcopy(new_strategy)
    
    antivax_history.append(sum(strategy.values()))

    if (random.random() < verbose):
      print("antivax at time ", t, ": ", sum(strategy.values())," Probability: ", sum(strategy.values()) / len(g.nodes) )
    new_strategy = simulate_one_parallel_epoch(graph = graph, x = strategy, alpha = alpha, beta = beta, gamma=gamma, C = C, delta = delta)

  return antivax_history, t, strategy

In [0]:
def get_init_sentiments(nnodes, PROB_PRO, PROB_ANTI, PROB_NEUTRAL):
  random.seed(42)
  initial_sentiments = np.random.choice(3, nnodes, p=[PROB_PRO, PROB_ANTI, PROB_NEUTRAL])

  init_sentiments_dict = {i: initial_sentiments[i] for i in range(len(initial_sentiments))}
  # print(init_sentiments_dict)

  return init_sentiments_dict

In [0]:
#### Experiment 1
#### this script plots histogram of number of epochs for NE
## varying the init strategy vector with fixed probabilities

RUN_THIS = False


alpha = 0
beta = 1

C_list = [70]
gamma_list = [0.9]
delta_list = [35]

PROB_PRO = 0.7
PROB_NEUTRAL = 0.0
PROB_ANTI = 0.3

NUMBER_OF_INIT_STRATEGY = 100

if RUN_THIS and __name__ == "__main__":

  epoch_list = []
  
  for gamma in gamma_list:
    for delta in delta_list:
      for C in C_list:
        for e in range(NUMBER_OF_INIT_STRATEGY):

          init_sentiments_dict = get_init_sentiments(g.number_of_nodes(), PROB_PRO=PROB_PRO, PROB_ANTI=PROB_ANTI, PROB_NEUTRAL = PROB_NEUTRAL)

          antivax_history, t, strategyNE = simulate_parallel(verbose = 0, graph=g, strategy= init_sentiments_dict, alpha = alpha, beta=beta, gamma=gamma, C = C, delta = delta) 
          epoch_list.append(t)
          # plot(x = range(len(antivax_history)), y = antivax_history, title = "gamma = {}, delta = {}, C = {}".format(gamma, delta, C), xlabel="# Epochs", ylabel="# Antivaccine nodes")


  plt.hist(epoch_list, bins = 'auto')

In [0]:
##### Experiment 2
##### plot # antivax per epoch

RUN_THIS = False

alpha = 5
beta = 1
# gamma = 0.9
# C = 70
# delta = 35 ## herd immunity does not guarantee total immunity
C_list = [0, 1, 8, 16, 32, 70]
gamma_list = [0.9]
delta_list = [35]

PROB_PRO = 0.7
PROB_NEUTRAL = 0.0
PROB_ANTI = 0.3

init_sentiments_dict = get_init_sentiments(g.number_of_nodes(), PROB_PRO=PROB_PRO, PROB_ANTI=PROB_ANTI, PROB_NEUTRAL = PROB_NEUTRAL)

if RUN_THIS and __name__ == "__main__":
  for gamma in gamma_list:
    for delta in delta_list:
      for C in C_list:
        antivax_history, t, strategyNE = simulate_parallel(verbose = 1, graph=g, strategy= init_sentiments_dict, alpha = alpha, beta=beta, gamma=gamma, C = C, delta = delta) 
        plot(showplot=False, x = range(len(antivax_history)), y = antivax_history, title="", label = "gamma = {}, delta = {}, C = {}".format(gamma, delta, C), xlabel="# Epochs", ylabel="# Antivaccine nodes")
        plt.legend()
        # plt.show()
        print("Time to converge:", t)

In [0]:
def get_antivax(strategyNE):
  s = []
  for i in range(len(strategyNE)):
    if (strategyNE[i] == 1):
      # print(i)
      s.append(i)
  
  return s

In [0]:
### Experiment 3
### let's understand the antivax nodes in NE
#############################################

RUN_THIS = True

alpha = 1
beta = 1

C_list = [2]
gamma_list = [0.9]
delta_list = [1]

PROB_PRO = 0.6
PROB_NEUTRAL = 0.0
PROB_ANTI = 0.4


NUM_ITERATION = 100

anti_degree_global_count = {} # dictionary of lists

if RUN_THIS and __name__ == "__main__":

  anti_degree_list = []
  anti_cluster_coef_list = []
  for gamma in gamma_list:
    for delta in delta_list:
      for C in C_list:
        anti_degree_local_count = {} # dictionary of ints

        for t in range(NUM_ITERATION):

          init_sentiments_dict = get_init_sentiments(g.number_of_nodes(), PROB_PRO=PROB_PRO, PROB_ANTI=PROB_ANTI, PROB_NEUTRAL = PROB_NEUTRAL)
          
          antivax_history, t, strategyNE = simulate_parallel(TMAX= 20,verbose = 0.01, graph=g, strategy= init_sentiments_dict, alpha = alpha, beta=beta, gamma=gamma, C = C, delta = delta) 
          # plot(x = range(len(antivax_history)), y = antivax_history, title = "gamma = {}, delta = {}, C = {}".format(gamma, delta, C), xlabel="# Epochs", ylabel="# Antivaccine nodes")
          antivaxNE = get_antivax(strategyNE)

          for i in antivaxNE:
            if not g.degree[i] in anti_degree_local_count.keys():
              anti_degree_local_count[g.degree[i]] = 0
            
            anti_degree_local_count[g.degree[i]] += 1
            anti_degree_list.append(g.degree[i])
            anti_cluster_coef_list.append(nx.clustering(g,i))
          
          # print(anti_degree_local_count)

          for deg, count in anti_degree_local_count.items():

            if not deg in anti_degree_global_count.keys():
              anti_degree_global_count[deg]=[]

            anti_degree_global_count[deg].append(count)


    save_obj(anti_degree_global_count, 'anti_degree_global_count')
    # plt.hist(anti_degree_list, bins = 'auto')
    # plt.hist(anti_cluster_coef_list, bins='auto')

antivax at time  0 :  1634  Probability:  0.40455558306511513
antivax at time  0 :  1616  Probability:  0.40009903441445904
antivax at time  20 :  66  Probability:  0.016340678385739046
antivax at time  0 :  1617  Probability:  0.4003466204506066
antivax at time  0 :  1581  Probability:  0.39143352314929436
antivax at time  20 :  75  Probability:  0.018568952711067097
antivax at time  0 :  1580  Probability:  0.3911859371131468
antivax at time  0 :  1620  Probability:  0.40108937855904925
antivax at time  0 :  1643  Probability:  0.4067838573904432
antivax at time  0 :  1661  Probability:  0.4112404060410993
antivax at time  0 :  1627  Probability:  0.4028224808120822
antivax at time  0 :  1605  Probability:  0.39737558801683587
antivax at time  0 :  1648  Probability:  0.40802178757118096
antivax at time  0 :  1626  Probability:  0.4025748947759346
antivax at time  20 :  97  Probability:  0.024015845506313444
antivax at time  0 :  1626  Probability:  0.4025748947759346
antivax at time

In [0]:
def 

{1: [2,
  2,
  2,
  2,
  2,
  2,
  2,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  16,
  18,
  18,
  18,
  18,
  18,
  18,
  18,
  18,
  18,
  18,
  18,
  18,
  18,
  18,
  18,
  18,
  18,
  18,
  18,
  18,
  18,
  18,
  18,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  25,
  32,
  32,
  32,
  34,
  34,
  34,
  34,
  34,
  34,
  34,
  34,
  34,
  34,
  34,
  34,
  34,
  34],
 2: [1,
  1,
  1,
  4,
  4,
  4,
  4,
  4,
  6,
  6,
  14,
  14,
  14,
  14,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  16,
  16,
  24,
  28,
  28,
  28,
  30,
  30,
  40,
  40,
  44,
  45,
  45,
  45,
  46,
  46,
  50,
  54,
  55,
  58,
  58,
  58,
  58,
  58,
  58,
  61,
  67,
  67,
  67,
  71,
  72,
  72,
  78,
  78,
  78,
  79,
  79,
  82,
  82,
  83,
  83,
  83,
  83,
  85,
  85,
  85,
  85,
  85,
  85,
  85,
  85,
  85,
  85,
  88,
  89,
  89,
  8

In [0]:
# plt.hist(anti_degree_list, bins = 'auto', label="Histogram of degree of Antivaccine nodes in NE")
# plt.hist(anti_cluster_coef_list, bins='auto')

def boxplot_dictionary_in_chunks(dict1, split):
  # labels, data = [*zip(*dict1.items())]  # 'transpose' items to parallel key, value lists

  # or backwards compatable    
  labels = [key for key in sorted(dict1.keys())]
  data =  [dict1[key] for key in sorted(dict1.keys())]

  chunk = len(data)//split

  for i in range(split):
    plt.boxplot(data[i* chunk : (i+1)*chunk])
    plt.xticks(range(1 , chunk+1), labels[i* chunk: (i+1)*chunk])
    plt.xlabel("Degree of nodes in NE")
    plt.ylabel("Number of nodes")
    plt.show()

    # fig = plt.gcf()
    # fig.set_size_inches(40, 5)
    # fig.savefig('boxplot.png', dpi=200)


  plt.boxplot(data[split* chunk : -1])
  plt.xticks(range(len(labels[split* chunk : -1])), labels[split* chunk : -1])
  plt.xlabel("Degree of nodes in NE")
  plt.ylabel("Number of nodes")
  plt.show()


RUN_THIS = False

if RUN_THIS:
  boxplot_dictionary_in_chunks(anti_degree_global_count, split=6)



In [0]:
def make_antivax(strategy, s):
  for i in s:
    strategy[i] = 1

  return strategy


def make_provax(strategy, s):
  for i in s:
    strategy[i] = 0

  return strategy

In [0]:
def is_subset(lst1, lst2):

  for i in lst1:
    if( i not in lst2):
      # print(i)
      return False

  return True

In [0]:
### connect subgraph and make them antivax
#########################################

RUN_THIS = False


k = 100
S = random.sample(range(g.number_of_nodes()), k)

for i in S:
  for j in S:
    if not g.has_edge(i,j):
      g.add_edge(i,j)


#### this script plots histogram of number of epochs for NE
## varying the init strategy vector with fixed probabilities

alpha = 0
beta = 1
# gamma = 0.9
# C = 70
# delta = 35 ## herd immunity does not guarantee total immunity
C_list = [0]
gamma_list = [0.9]
delta_list = [35]

PROB_PRO = 0.7
PROB_NEUTRAL = 0.0
PROB_ANTI = 0.3

if RUN_THIS and __name__ == "__main__":

  epoch_list = []
  
  for gamma in gamma_list:
    for delta in delta_list:
      for C in C_list:


        init_sentiments_dict = get_init_sentiments(g.number_of_nodes(), PROB_PRO=PROB_PRO, PROB_ANTI=PROB_ANTI, PROB_NEUTRAL = PROB_NEUTRAL)

        make_antivax(init_sentiments_dict, S)

        antivax_history, t, strategyNE = simulate_parallel(verbose = 1, graph=g, strategy= init_sentiments_dict, alpha = alpha, beta=beta, gamma=gamma, C = C, delta = delta) 

        plot(x = range(len(antivax_history)), y = antivax_history, title = "gamma = {}, delta = {}, C = {}".format(gamma, delta, C), xlabel="# Epochs", ylabel="# Antivaccine nodes")


  print( is_subset( S, get_antivax(strategyNE=strategyNE) ) )
  print(S)
  print(get_antivax( strategyNE= strategyNE))

In [0]:
## Experiment 4
### stubborn nodes
### do it randomly
#########################################

RUN_THIS = True


k = 100
S = random.sample(range(g.number_of_nodes()), k)


